# JapDic 
Version 2.0
Made by Vik

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from notion_client import Client
import os
from dotenv import load_dotenv

KeyboardInterrupt: 

In [2]:
load_dotenv("locals.env")
token = os.getenv("NOTION_TOKEN")

In [3]:
# Read the words from the text file
with open("data/18_11_list.txt", "r") as f:
    for line in f:
        words = line.strip().split(",")
        english_word = words[0]
        romaji_word = words[1]
        page_title = f"{english_word} - {romaji_word}"

In [4]:
# Replace with your Notion integration token and database ID
token = os.getenv("NOTION_TOKEN")
# database_id = os.getenv("NOTION_DATABASE_ID")
page_id = os.getenv("NOTION_JAPDIC_DEMO_PAGE_ID")

client = Client(auth=token)

print(page_id)

142ebe12-09f2-807c-aa24-e597aa2ec4de


In [5]:
page = client.pages.retrieve(page_id=page_id)

# print(page)

{'object': 'page', 'id': '142ebe12-09f2-807c-aa24-e597aa2ec4de', 'created_time': '2024-11-18T23:37:00.000Z', 'last_edited_time': '2024-11-18T23:38:00.000Z', 'created_by': {'object': 'user', 'id': 'bd435e00-772b-4052-9eb6-e9304ec07464'}, 'last_edited_by': {'object': 'user', 'id': 'bd435e00-772b-4052-9eb6-e9304ec07464'}, 'cover': None, 'icon': None, 'parent': {'type': 'page_id', 'page_id': '128ebe12-09f2-8078-9eb4-d036d942ba10'}, 'archived': False, 'in_trash': False, 'properties': {'title': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'JapDic - demo 2', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'JapDic - demo 2', 'href': None}]}}, 'url': 'https://www.notion.so/JapDic-demo-2-142ebe1209f2807caa24e597aa2ec4de', 'public_url': None, 'request_id': 'b2c3880f-3e18-46b5-ac1d-dbe9285ceb22'}


## Add header case

In [6]:
header_block = {
    "object": "block",
    "type": "header",
    "header": {"text": [{"text": {"content": "This is a header"}}]},
}

# response = client.pages.update(page_id=page_id, children=[header_block])
# print(response)

{'object': 'page', 'id': '142ebe12-09f2-807c-aa24-e597aa2ec4de', 'created_time': '2024-11-18T23:37:00.000Z', 'last_edited_time': '2024-11-18T23:38:00.000Z', 'created_by': {'object': 'user', 'id': 'bd435e00-772b-4052-9eb6-e9304ec07464'}, 'last_edited_by': {'object': 'user', 'id': 'bd435e00-772b-4052-9eb6-e9304ec07464'}, 'cover': None, 'icon': None, 'parent': {'type': 'page_id', 'page_id': '128ebe12-09f2-8078-9eb4-d036d942ba10'}, 'archived': False, 'in_trash': False, 'properties': {'title': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'JapDic - demo 2', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'JapDic - demo 2', 'href': None}]}}, 'url': 'https://www.notion.so/JapDic-demo-2-142ebe1209f2807caa24e597aa2ec4de', 'public_url': None, 'request_id': 'e58477a6-5f65-4791-8921-7b549131d229'}


In [2]:
# Function to add text to a page
def add_text_to_page(page_id: str, text: str):
    """
    Adds text as a paragraph block to a Notion page

    Args:
        page_id (str): The ID of the Notion page
        text (str): The text content to add
    """
    try:
        response = client.blocks.children.append(
            block_id=page_id,
            children=[
                {
                    "type": "paragraph",
                    "paragraph": {
                        "rich_text": [
                            {"type": "text", "text": {"content": text, "link": None}}
                        ],
                        "color": "default",
                    },
                }
            ],
        )
        return response
    except Exception as e:
        print(f"Error adding text to page: {e}")
        return None


# Example usage
# page_id = "YOUR_PAGE_ID"  # Replace with your page ID
add_text_to_page(page_id, "demo text")

Error adding text to page: name 'client' is not defined


In [31]:
# Test function to verify word parsing
def test_parse_word_line():
    # Test case 1: Normal case
    test_line = "hello,konnichiwa"
    english, romaji = test_line.strip().split(",")
    assert english == "hello"
    assert romaji == "konnichiwa"

    # Test case 2: With spaces
    test_line = "good morning, ohayou gozaimas"
    english, romaji = test_line.strip().split(",")
    assert english == "good morning"
    assert romaji == " ohayou gozaimas"

    # Test case 3: Empty values
    test_line = ","
    english, romaji = test_line.strip().split(",")
    assert english == ""
    assert romaji == ""


# Run the test
test_parse_word_line()
print("All tests passed!")


All tests passed!
